In [1]:
import keras
import wandb
from wandb.keras import WandbCallback
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
import DatasetParser
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.
wandb: WARNING Keras version 2.3.1 is not fully supported. Required keras >= 2.4.0


In [2]:
(x_train, y_train_raw), (x_test, y_test_raw), class_names = DatasetParser.load_data("Resnet")

224
Entrenamiento: 204
Test: 80


In [3]:
# One hot encode ouput
y_train = keras.utils.to_categorical(y_train_raw)
y_test = keras.utils.to_categorical(y_test_raw)

In [4]:
# Load ResNet50 Trained on imagenet
resnet_model = ResNet50(weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=True)


In [5]:
from keras.preprocessing import image
import numpy as np
img = image.load_img('elephant.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = resnet_model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n01871265', 'tusker', 0.48542696), ('n02504013', 'Indian_elephant', 0.3432928), ('n02504458', 'African_elephant', 0.17125095)]


In [6]:
# We should preprocess the images the same way resnet images were preprocessed
x_train_preprocessed = preprocess_input(x_train)
x_test_preprocessed = preprocess_input(x_test)

In [7]:
# Build a new model that is ResNet50 minus the very last layer
last_layer = resnet_model.get_layer("avg_pool")

resnet_layers = keras.Model(inputs=resnet_model.inputs, outputs=last_layer.output)
resnet_layers.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [8]:
# We use our resnet to "predict" but because we have removed the top layer, 
# this outputs the activations of the second to last layer on our dataset

x_train_features = resnet_layers.predict(x_train_preprocessed)

In [9]:
x_test_features = resnet_layers.predict(x_test_preprocessed)

In [10]:
# We can directly stich the models together

Fine_Tuning_model=Sequential()
Fine_Tuning_model.add(resnet_layers)
Fine_Tuning_model.add(Dense(2, activation="sigmoid"))

Fine_Tuning_model.layers[0].trainable=False

Fine_Tuning_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Fine_Tuning_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [11]:
wandb.init(project="tf-covid-19")
Fine_Tuning_model.fit(x_train_preprocessed, y_train, epochs=15, validation_data=(x_test_preprocessed, y_test), callbacks=[WandbCallback()])

wandb: Currently logged in as: scueroinc (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Train on 204 samples, validate on 80 samples
Epoch 1/15
204/204 [==============================] - 5s 25ms/step - loss: 0.6070 - accuracy: 0.6961 - val_loss: 0.4276 - val_accuracy: 0.7875
Epoch 2/15
204/204 [==============================] - 3s 13ms/step - loss: 0.2438 - accuracy: 0.9804 - val_loss: 0.5636 - val_accuracy: 0.7375
Epoch 3/15
204/204 [==============================] - 3s 13ms/step - loss: 0.0965 - accuracy: 0.9853 - val_loss: 0.3391 - val_accuracy: 0.8125
Epoch 4/15
204/204 [==============================] - 3s 13ms/step - loss: 0.0462 - accuracy: 0.9902 - val_loss: 0.3984 - val_accuracy: 0.8125
Epoch 5/15
204/204 [==============================] - 3s 13ms/step - loss: 0.0343 - accuracy: 0.9902 - val_loss: 0.4974 - val_accuracy: 0.7875
Epoch 6/15
204/204 [==============================] - 3s 13ms/step - loss: 0.0335 - accuracy: 0.9902 - val_loss: 0.3721 - val_accuracy: 0.8250
Epoch 7/15
204/204 [==============================] - 3s 13ms/step - loss: 0.0377 - accuracy: 0.9

In [12]:
# We can allow some of the resnet layers to change as we train.  
# Typically you would want to lower the learning rate in conjunction with this.

Fine_Tuning_model.layers[0].trainable = True

# We let the last 3 blocks train
for layer in Fine_Tuning_model.layers[0].layers[:-11]:
    layer.trainable = False
for layer in Fine_Tuning_model.layers[0].layers[-11:]:
    layer.trainable = True
    
Fine_Tuning_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
Fine_Tuning_model.fit(x_train_preprocessed, y_train, epochs=15, validation_data=(x_test_preprocessed, y_test), callbacks=[WandbCallback()])

Train on 204 samples, validate on 80 samples
Epoch 1/15
204/204 [==============================] - 7s 36ms/step - loss: 0.0113 - accuracy: 0.9951 - val_loss: 4.6245 - val_accuracy: 0.5875
Epoch 2/15
204/204 [==============================] - 3s 14ms/step - loss: 0.0523 - accuracy: 0.9853 - val_loss: 5.4204 - val_accuracy: 0.5875
Epoch 3/15
204/204 [==============================] - 3s 14ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 7.0097 - val_accuracy: 0.5375
Epoch 4/15
204/204 [==============================] - 3s 14ms/step - loss: 0.0281 - accuracy: 0.9853 - val_loss: 5.7001 - val_accuracy: 0.5875
Epoch 5/15
204/204 [==============================] - 3s 14ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 3.1949 - val_accuracy: 0.7500
Epoch 6/15
204/204 [==============================] - 3s 14ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 2.1500 - val_accuracy: 0.8500
Epoch 7/15
204/204 [==============================] - 3s 14ms/step - loss: 9.8432e-04 - accuracy: